In [1]:
import re
import collections
import math
import xlrd
import xlsxwriter
import numpy as np
import datetime
import codecs
from scipy.sparse import csr_matrix
from scipy.sparse import csc_matrix
from scipy.spatial.distance import cdist

In [2]:
def add_months(s, p):
    ''' 在日期s上加上p个月，如果日期不存在，顺延到下个月1号 '''
    p_years = s.year + p / 12
    p_months = s.month + p % 12 
    if p_months > 12:
        p_years += 1
        p_months = p_months % 12
    
    try:
        rs = datetime.datetime(p_years, p_months, s.day)
    except ValueError, e:
        if p_months == 12:
            rs = datetime.datetime(p_years + 1, 1, 1)
        else:
            rs = datetime.datetime(p_years, p_months+1, 1)
    return rs

def next_quater(cur_quater):
    return add_months(cur_quater, 3)

class SharesInfo(object):
    def __init__(self, shares, marketValue, proportion):
        self.shares_ = shares
        self.marketValue_ = marketValue
        self.proportion_ = proportion
    
    def __str__(self):
        return "%f %f %f"%(self.shares_, self.marketValue_, self.proportion_)
    def __repr__(self):
        return "%f %f %f"%(self.shares_, self.marketValue_, self.proportion_)


# 季度的分割日期，[datetime.datetime(2004, 1, 1, 0, 0), datetime.datetime(2004, 4, 1, 0, 0), datetime.datetime(2004, 7, 1, 0, 0) 。。。
quater_date = [datetime.datetime(2003,7,1),datetime.datetime(2003,10,1)] \
        + [datetime.datetime(v1,v2,1) for v1 in xrange(2004, 2018) for v2 in xrange(1,12,3)]

In [3]:
g_raw_data = collections.defaultdict(lambda : collections.defaultdict(dict))
g_record = 0
g_stock_set = set()
def load_data(file_name="Fund_Portfolio_Stock2003-2007.txt"):
    global g_record
    with codecs.open(file_name, "r", encoding="utf_16") as ifid:
        line_num = 0
        for line in ifid:
            if line_num > 0 and len(line) > 5:
                try:
                    MasterFundCode, ReportTypeID, Startdate, EndDate, Rank, Symbol, StockName, Shares, MarketValue, Proportion \
                        = line.strip().split('\t')
                    Startdate = datetime.datetime.strptime(Startdate, "%Y-%m-%d")
                    EndDate = datetime.datetime.strptime(EndDate, "%Y-%m-%d")
                    if int(ReportTypeID) in (1,2,3,4) and Startdate >= quater_date[0] and EndDate < quater_date[-1]:
                        real_start_date = datetime.datetime(Startdate.year, 3*int(ReportTypeID)-2, 1)
                        real_idx = quater_date.index(real_start_date)
                        
                        if quater_date[real_idx] <= Startdate < quater_date[real_idx+1] \
                            and quater_date[real_idx] < EndDate < quater_date[real_idx+1] \
                            and Startdate <= EndDate:
                            g_raw_data[MasterFundCode][Symbol][real_start_date] = SharesInfo(float(Shares), float(MarketValue), float(Proportion))
                            g_stock_set.add(Symbol)
                            g_record += 1
                        else:
                            print u"error at %s linenum %d:%s %s %s"  %(file_name, line_num+1, MasterFundCode, Symbol, Startdate)
                except ValueError, e:
                    print line.strip(), file_name, line_num+1
            line_num += 1


                            
load_data("Fund_Portfolio_Stock2003-2007.txt")
load_data("Fund_Portfolio_Stock2008-2012.txt")
load_data("Fund_Portfolio_Stock2013-2017.txt")
print "加载%d只基金%d条数据，共%d只股票" %(len(g_raw_data), g_record, len(g_stock_set))


040001	3	2003-07-01	2003-09-30	5	600018	上港集箱		121492440.00	3.8900 Fund_Portfolio_Stock2003-2007.txt 4763
040001	1	2004-01-01	2004-03-31	10	600018	上港集箱		52769378.60	2.0300 Fund_Portfolio_Stock2003-2007.txt 4779
050001	3	2003-07-01	2003-09-30	3	600018	上港集箱		150526740.00	6.8900 Fund_Portfolio_Stock2003-2007.txt 6108
070001	3	2003-07-01	2003-09-30	8	600018	上港集箱		44455572.00	4.0100 Fund_Portfolio_Stock2003-2007.txt 8027
070003	3	2003-07-01	2003-09-30	2	600018	上港集箱		36853884.00	3.7200 Fund_Portfolio_Stock2003-2007.txt 9434
090001	1	2004-01-01	2004-03-31	10	600018	上港集箱		27356355.80	2.4500 Fund_Portfolio_Stock2003-2007.txt 12417
162203	3	2003-07-01	2003-09-30	2	600018	上港集箱		17040000.00	3.3200 Fund_Portfolio_Stock2003-2007.txt 28501
213001	3	2003-07-01	2003-09-30	5	600018	上港集箱		27635940.00	3.8600 Fund_Portfolio_Stock2003-2007.txt 35658
217001	3	2003-07-01	2003-09-30	1	600018	上港集箱		47695128.00	3.3100 Fund_Portfolio_Stock2003-2007.txt 36656
217001	1	2004-01-01	2004-03-31	6	600018	上港集箱		46939204.4

In [ ]:
#dump g_stock_map
with xlsxwriter.Workbook("quarter_stock_list.xlsx") as wb:
    ws = wb.add_worksheet()
    wt_idx = 0
    for stock_name in sorted(g_stock_set):
        ws.write(wt_idx, 0, stock_name)
        wt_idx += 1

In [4]:
g_holding_num_data = collections.defaultdict(lambda : collections.defaultdict(lambda : np.zeros(len(g_stock_set))))
g_sorted_stock = sorted(g_stock_set)
g_stock_map = {sym_: i_ for i_, sym_ in enumerate(sorted(g_stock_set))}
for fund_ in g_raw_data:
    for sym_ in g_raw_data[fund_]:
        for cur_ in g_raw_data[fund_][sym_]:
            g_holding_num_data[fund_][cur_][g_stock_map[sym_]] = g_raw_data[fund_][sym_][cur_].shares_


In [ ]:
# dump 持股变化率，注意，执行过后会对g_holding_num_data的结构造成影响，必须重新生成g_holding_num_data
with xlsxwriter.Workbook("share_change_rate.xlsx") as wb:
    ws = wb.add_worksheet()
    ws.write(0,0,"fund")
    ws.write(0,1,"quarter")
    ws.write(0,2,"stock")
    ws.write(0,3,"change rate")
    wt_idx = 1
    for fund_ in g_holding_num_data:
        start_date_ = sorted(g_holding_num_data[fund_].keys())[0]
        end_date_ = sorted(g_holding_num_data[fund_].keys())[-1]
        for i in xrange(quater_date.index(start_date_), quater_date.index(end_date_)+1):
            for j, stock_name in enumerate(g_sorted_stock):
                previous_share = g_holding_num_data[fund_][quater_date[i-1]][j]
                current_share = g_holding_num_data[fund_][quater_date[i]][j]
                
                if previous_share == 0.0 and current_share == 0.0:
                    continue
                ws.write(wt_idx,0,fund_)
                ws.write(wt_idx,1,u"%dQ%d"%(quater_date[i].year, (quater_date[i].month+2)/3))
                ws.write(wt_idx,2,stock_name)
                out_value = 0.0
                if previous_share != 0.0 and current_share != 0.0:
                    out_value = (current_share-previous_share)/previous_share
                elif current_share == 0.0:
                    out_value = -1.0
                else:
                    out_value = 1.0
                ws.write(wt_idx,3,out_value)
                wt_idx += 1


In [5]:
g_trade_vector = collections.defaultdict(dict)
for fund_ in g_holding_num_data:
    start_date_ = sorted(g_holding_num_data[fund_].keys())[0]
    end_date_ = sorted(g_holding_num_data[fund_].keys())[-1]
    for i in xrange(quater_date.index(start_date_), quater_date.index(end_date_)+1):
        if quater_date[i] in g_holding_num_data[fund_]:
            if quater_date[i-1] in g_holding_num_data[fund_]:
                g_trade_vector[quater_date[i]][fund_] = \
                    g_holding_num_data[fund_][quater_date[i]] - g_holding_num_data[fund_][quater_date[i-1]]
            else:
                g_trade_vector[quater_date[i]][fund_] = g_holding_num_data[fund_][quater_date[i]]
        else:
            print u"error at %s %s"%(fund_, quater_date[i])

error at 002577 2017-01-01 00:00:00
error at 164205 2017-04-01 00:00:00
error at 671010 2015-07-01 00:00:00
error at 163001 2015-07-01 00:00:00
error at 202003 2017-01-01 00:00:00
error at 160133 2017-01-01 00:00:00
error at 001421 2017-01-01 00:00:00
error at 000527 2017-01-01 00:00:00
error at 202002 2017-01-01 00:00:00
error at 202009 2017-01-01 00:00:00
error at 160106 2017-01-01 00:00:00
error at 519697 2011-10-01 00:00:00
error at 160105 2017-01-01 00:00:00
error at 000554 2017-01-01 00:00:00
error at 080007 2013-10-01 00:00:00
error at 090013 2014-01-01 00:00:00
error at 000955 2017-01-01 00:00:00
error at 166301 2015-10-01 00:00:00
error at 002851 2017-01-01 00:00:00
error at 001181 2017-01-01 00:00:00
error at 202023 2017-01-01 00:00:00
error at 001162 2015-07-01 00:00:00
error at 001667 2017-01-01 00:00:00
error at 002160 2017-01-01 00:00:00
error at 080015 2012-10-01 00:00:00
error at 080015 2013-10-01 00:00:00
error at 320015 2014-01-01 00:00:00
error at 001053 2017-01-01 0

In [ ]:
print g_holding_num_data[u'000011'][datetime.datetime(2004,10,1)]
print g_trade_vector[datetime.datetime(2004,10,1)][u'000011']

In [6]:
g_rank_raw_data = collections.defaultdict(dict)
g_fund_set = set()
with xlrd.open_workbook(u'pref_order_result.xlsx') as wb:
    ws = wb.sheet_by_index(1)
    for r in xrange(1, ws.nrows):
        fund_id = ws.cell(r, 1).value
        quarter = datetime.datetime(int(ws.cell(r,2).value[:4]), int(ws.cell(r,2).value[5])*3-2, 1)
        order = float(ws.cell(r,3).value)
        
        if quarter in quater_date:
            g_rank_raw_data[quarter][fund_id] = order
            g_fund_set.add(fund_id)
        else:
            print "error at %s %s %f"%(fund_id, quarter, order)
            
g_sorted_fund = sorted(g_fund_set)
g_fund_map = {fund: i for i, fund in enumerate(g_sorted_fund)}
g_rank = collections.defaultdict(lambda : np.empty(len(g_sorted_fund), dtype='int64'))
g_funds_num = collections.defaultdict(int)
g_rank_value = {}
tmp_order = collections.defaultdict(lambda : np.full(len(g_sorted_fund), -1.0))

for quarter in g_rank_raw_data:
    for fund in g_rank_raw_data[quarter]:
        tmp_order[quarter][g_fund_map[fund]] = g_rank_raw_data[quarter][fund]
        g_funds_num[quarter] += 1
        
for quarter in tmp_order:
    sorted_idx = np.argsort(tmp_order[quarter])[::-1]
    g_rank_value[quarter] = np.sort(tmp_order[quarter])[::-1]
    g_rank[quarter][sorted_idx] = np.arange(len(g_sorted_fund))

In [ ]:
print g_rank[datetime.datetime(2003,7,1)][g_fund_map[u'080001']]
print g_rank_raw_data[datetime.datetime(2003,7,1)][u'080001']
print g_funds_num[datetime.datetime(2004,10,1)]
print int(math.ceil(g_funds_num[datetime.datetime(2004,10,1)]*0.1))
print g_rank_value[datetime.datetime(2004,10,1)][:11][6]

In [ ]:
def intersection_cosine(a, b):
    flag=(a!=0)&(b!=0)
    if any(flag):
        return 1-np.inner(a,b)/(np.linalg.norm(a[flag])*np.linalg.norm(b[flag]))
    else:
        return 1

def calc_sim2(trade_vec):
    
    leading_sim_ = collections.defaultdict(dict)
    current_sim_ = collections.defaultdict(dict)
    following_sim_ = collections.defaultdict(dict)
    following_advance_sim_ = collections.defaultdict(dict)
    following_advance_5_sim_ = collections.defaultdict(dict)
    following_advance_10_sim_ = collections.defaultdict(dict)
    following_advance_15_sim_ = collections.defaultdict(dict)
    
    start_idx_ = quater_date.index(datetime.datetime(2004,1,1))
    end_idx_ = quater_date.index(datetime.datetime(2017,10,1))
    
    for i in xrange(start_idx_, end_idx_):
        current_date = quater_date[i]
        #print "calc %s"%(current_date)
        # 去除变化为0的funds，并且排序
        sorted_current_funds = sorted([funds_ for funds_ in trade_vec[current_date] if trade_vec[current_date][funds_].sum()!=0])
        current_matrix = np.matrix([trade_vec[current_date][funds_] for funds_ in sorted_current_funds])
        current_cos_dis = cdist(current_matrix, current_matrix, intersection_cosine)        
        
        print current_date
        
        for i_, fund_ in enumerate(sorted_current_funds):
            current_sim_[fund_][current_date] = (current_cos_dis[i_].sum()-current_cos_dis[i_][i_], len(sorted_current_funds) - 1)
    
    
    cross_start_idx = quater_date.index(datetime.datetime(2004,1,1))
    cross_end_idx = quater_date.index(datetime.datetime(2017,10,1))
    for i in xrange(cross_start_idx, cross_end_idx):
        previous_date = quater_date[i-1]
        current_date= quater_date[i]
        
        sorted_previous_funds = sorted([funds_ for funds_ in trade_vec[previous_date] if trade_vec[previous_date][funds_].sum()!=0])
        sorted_current_funds = sorted([funds_ for funds_ in trade_vec[current_date] if trade_vec[current_date][funds_].sum()!=0])
        
        previous_matrix = np.matrix([trade_vec[previous_date][funds_] for funds_ in sorted_previous_funds])
        current_matrix = np.matrix([trade_vec[current_date][funds_] for funds_ in sorted_current_funds])
        
        cross_cos_dis = cdist(previous_matrix, current_matrix, intersection_cosine)
        
        print current_date
        
        #print "calc cross %s %s, %s %s, %s"%(previous_date, current_date, previous_matrix.shape, current_matrix.shape, cross_cos_dis.shape)
        
        for i_, fund_ in enumerate(sorted_previous_funds):
            if fund_ in sorted_current_funds:
                remove_idx_ = sorted_current_funds.index(fund_) # 减去和T中自己的cos距离
                leading_sim_[fund_][previous_date] = (cross_cos_dis[i_].sum()-cross_cos_dis[i_][remove_idx_], len(sorted_current_funds) - 1)
            else:
                leading_sim_[fund_][previous_date] = (cross_cos_dis[i_].sum(), len(sorted_current_funds))
            
        for i_, fund_ in enumerate(sorted_current_funds):
            if fund_ in sorted_previous_funds:
                remove_idx_ =sorted_previous_funds.index(fund_)
                following_sim_[fund_][current_date] = (cross_cos_dis.T[i_].sum()-cross_cos_dis.T[i_][remove_idx_], len(sorted_previous_funds)-1)
            else:
                following_sim_[fund_][current_date] = (cross_cos_dis.T[i_].sum(), len(sorted_previous_funds))
                
        for i_, fund_ in enumerate(sorted_current_funds):
            order_ = g_rank_raw_data[previous_date].get(fund_, -1.0)
            sorted_previous_advance_flag = [True if g_rank_raw_data[previous_date].get(foo_fund, -2.0) > order_ else False \
                                         for foo_fund in sorted_previous_funds]
            if any(sorted_previous_advance_flag):
                following_advance_sim_[fund_][current_date] = (cross_cos_dis.T[i_][sorted_previous_advance_flag].sum(), \
                                                          sum(sorted_previous_advance_flag))
                
        for i_, fund_ in enumerate(sorted_current_funds):
            my_order_ = g_rank_raw_data[previous_date].get(fund_, -1.0)
            top10_num_ = int(math.ceil(g_funds_num[previous_date] * 0.1))
            top10_order_ = g_rank_value[previous_date][top10_num_]
            order_ = top10_order_ if top10_order_ > my_order_ else my_order_
            
            sorted_previous_advance_flag = [True if g_rank_raw_data[previous_date].get(foo_fund, -2.0) > order_ else False \
                                         for foo_fund in sorted_previous_funds]
            if any(sorted_previous_advance_flag):
                following_advance_10_sim_[fund_][current_date] = (cross_cos_dis.T[i_][sorted_previous_advance_flag].sum(), \
                                                          sum(sorted_previous_advance_flag))
                
        for i_, fund_ in enumerate(sorted_current_funds):
            my_order_ = g_rank_raw_data[previous_date].get(fund_, -1.0)
            top10_num_ = int(math.ceil(g_funds_num[previous_date] * 0.05))
            top10_order_ = g_rank_value[previous_date][top10_num_]
            order_ = top10_order_ if top10_order_ > my_order_ else my_order_
            
            sorted_previous_advance_flag = [True if g_rank_raw_data[previous_date].get(foo_fund, -2.0) > order_ else False \
                                         for foo_fund in sorted_previous_funds]
            if any(sorted_previous_advance_flag):
                following_advance_5_sim_[fund_][current_date] = (cross_cos_dis.T[i_][sorted_previous_advance_flag].sum(), \
                                                          sum(sorted_previous_advance_flag))

        for i_, fund_ in enumerate(sorted_current_funds):
            my_order_ = g_rank_raw_data[previous_date].get(fund_, -1.0)
            top10_num_ = int(math.ceil(g_funds_num[previous_date] * 0.15))
            top10_order_ = g_rank_value[previous_date][top10_num_]
            order_ = top10_order_ if top10_order_ > my_order_ else my_order_
            
            sorted_previous_advance_flag = [True if g_rank_raw_data[previous_date].get(foo_fund, -2.0) > order_ else False \
                                         for foo_fund in sorted_previous_funds]
            if any(sorted_previous_advance_flag):
                following_advance_15_sim_[fund_][current_date] = (cross_cos_dis.T[i_][sorted_previous_advance_flag].sum(), \
                                                          sum(sorted_previous_advance_flag))
                
            if fund_ == u'000001' and previous_date == datetime.datetime(2004,10,1):
                print sorted_previous_funds
                print sorted_previous_advance_flag
                print trade_vec[previous_date].keys()
                print [trade_vec[previous_date][funds_].sum() for funds_ in trade_vec[previous_date]]
                
                
    return leading_sim_, current_sim_, following_sim_, following_advance_sim_, \
                following_advance_5_sim_, following_advance_10_sim_, following_advance_15_sim_

g_leading_sim, g_current_sim, g_following_sim, g_following_advance_sim, \
    g_following_advance_5_sim, g_following_advance_10_sim, g_following_advance_15_sim = calc_sim2(g_trade_vector)

In [19]:
jingli_id_info = collections.defaultdict(dict)
jingli_id_name_set = set()

with xlrd.open_workbook(u"稳健型检验.xlsx") as wb:
    ws = wb.sheet_by_index(0)
    for r in xrange(1, ws.nrows):
        funds = ws.cell(r,1).value
        quater = datetime.datetime(int(ws.cell(r,2).value[:4]), 3*int(ws.cell(r,2).value[5])-2, 1)
        name = ws.cell(r,3).value
        jingli_id_name_set.add(name)
        if quater not in quater_date:
            print u"error at %s %s"%(funds, quater)
        else:            
            jingli_id_info[funds][quater] = name

jingli_id_name_map = {name_: i_ for i_, name_ in enumerate(sorted(jingli_id_name_set)) }
print jingli_id_name_map[u'陈仕德']
print jingli_id_info

1127
defaultdict(<type 'dict'>, {u'000828': {datetime.datetime(2017, 4, 1, 0, 0): u'\u5f20\u52cb', datetime.datetime(2016, 10, 1, 0, 0): u'\u5f20\u52cb', datetime.datetime(2017, 7, 1, 0, 0): u'\u5f20\u52cb', datetime.datetime(2016, 1, 1, 0, 0): u'\u5f20\u52cb', datetime.datetime(2016, 7, 1, 0, 0): u'\u5f20\u52cb', datetime.datetime(2017, 1, 1, 0, 0): u'\u5f20\u52cb', datetime.datetime(2015, 7, 1, 0, 0): u'\u5434\u534e', datetime.datetime(2016, 4, 1, 0, 0): u'\u5f20\u52cb', datetime.datetime(2015, 4, 1, 0, 0): u'\u5434\u534e', datetime.datetime(2015, 10, 1, 0, 0): u''}, u'110001': {datetime.datetime(2004, 10, 1, 0, 0): u'\u6c5f\u4f5c\u826f', datetime.datetime(2016, 10, 1, 0, 0): u'\u9648\u7693', datetime.datetime(2013, 4, 1, 0, 0): u'\u9648\u7693', datetime.datetime(2014, 10, 1, 0, 0): u'\u9648\u7693', datetime.datetime(2005, 4, 1, 0, 0): u'\u6c5f\u4f5c\u826f', datetime.datetime(2013, 10, 1, 0, 0): u'\u9648\u7693', datetime.datetime(2011, 4, 1, 0, 0): u'\u4faf\u6e05\u6fef', datetime.dat

In [27]:
SPECIAL_RET = 10000.0

def not_myself_cosine(a, b):
    if a[-1] != b[-1]:
        return 1-np.inner(a[:-1],b[:-1])/(np.linalg.norm(a[:-1])*np.linalg.norm(b[:-1]))
    else:
        return SPECIAL_RET

def calc_sim3(trade_vec):
    
    leading_sim_ = collections.defaultdict(dict)
    current_sim_ = collections.defaultdict(dict)
    following_sim_ = collections.defaultdict(dict)
    following_advance_sim_ = collections.defaultdict(dict)
    following_advance_5_sim_ = collections.defaultdict(dict)
    following_advance_10_sim_ = collections.defaultdict(dict)
    following_advance_15_sim_ = collections.defaultdict(dict)
    
    start_idx_ = quater_date.index(datetime.datetime(2004,1,1))
    end_idx_ = quater_date.index(datetime.datetime(2017,10,1))
    
#     for i in xrange(start_idx_, end_idx_):
#         current_date = quater_date[i]
#         sorted_current_funds = sorted([funds_ for funds_ in trade_vec[current_date] if trade_vec[current_date][funds_].sum()!=0])
#         for fund_ in sorted_current_funds:
#             if current_date not in jingli_id_info[fund_]:
#                 print fund_, current_date
                
#     return
    
    for i in xrange(start_idx_, end_idx_):
        current_date = quater_date[i]
        #print "calc %s"%(current_date)
        # 去除变化为0的funds，并且排序
        sorted_current_funds = sorted([funds_ for funds_ in trade_vec[current_date] 
                                       if trade_vec[current_date][funds_].sum()!=0 
                                       and current_date in jingli_id_info[funds_]])
        
        current_matrix = np.matrix([np.append(trade_vec[current_date][funds_], jingli_id_name_map[jingli_id_info[funds_][current_date]]) 
                                    for funds_ in sorted_current_funds])
        current_cos_dis = cdist(current_matrix, current_matrix, not_myself_cosine)        
        
        print current_date
        
        for i_, fund_ in enumerate(sorted_current_funds):
            not_special_flag = [True if this_cos_value != SPECIAL_RET else False for this_cos_value in current_cos_dis[i_]]
            current_sim_[fund_][current_date] = (current_cos_dis[i_][not_special_flag].sum(), sum(not_special_flag))
    
    
    cross_start_idx = quater_date.index(datetime.datetime(2004,1,1))
    cross_end_idx = quater_date.index(datetime.datetime(2017,10,1))
    for i in xrange(cross_start_idx, cross_end_idx):
        previous_date = quater_date[i-1]
        current_date= quater_date[i]
        
        sorted_previous_funds = sorted([funds_ for funds_ in trade_vec[previous_date] 
                                        if trade_vec[previous_date][funds_].sum()!=0 and 
                                       previous_date in jingli_id_info[funds_]])
        sorted_current_funds = sorted([funds_ for funds_ in trade_vec[current_date] 
                                       if trade_vec[current_date][funds_].sum()!=0 and
                                      current_date in jingli_id_info[funds_]])
        
        if not (sorted_previous_funds and sorted_current_funds):
            continue
        
        previous_matrix = np.matrix([np.append(trade_vec[previous_date][funds_], jingli_id_name_map[jingli_id_info[funds_][previous_date]])
                                     for funds_ in sorted_previous_funds])
        current_matrix = np.matrix([np.append(trade_vec[current_date][funds_], jingli_id_name_map[jingli_id_info[funds_][current_date]])
                                    for funds_ in sorted_current_funds])
        
        print current_date, previous_matrix.shape, current_matrix.shape
        cross_cos_dis = cdist(previous_matrix, current_matrix, not_myself_cosine)
        
        #print "calc cross %s %s, %s %s, %s"%(previous_date, current_date, previous_matrix.shape, current_matrix.shape, cross_cos_dis.shape)
        
        for i_, fund_ in enumerate(sorted_previous_funds):
            not_special_flag = [True if this_cos_value != SPECIAL_RET else False for this_cos_value in cross_cos_dis[i_]]
            leading_sim_[fund_][previous_date] = (cross_cos_dis[i_][not_special_flag].sum(), sum(not_special_flag))
            
        for i_, fund_ in enumerate(sorted_current_funds):
            not_special_flag = [True if this_cos_value != SPECIAL_RET else False for this_cos_value in cross_cos_dis.T[i_]]
            following_sim_[fund_][current_date] = (cross_cos_dis.T[i_][not_special_flag].sum(), sum(not_special_flag))
                
        for i_, fund_ in enumerate(sorted_current_funds):
            order_ = g_rank_raw_data[previous_date].get(fund_, -1.0)
            sorted_previous_advance_flag = [True if g_rank_raw_data[previous_date].get(foo_fund, -2.0) > order_ else False \
                                         for foo_fund in sorted_previous_funds]
            if any(sorted_previous_advance_flag):
                following_advance_sim_[fund_][current_date] = (cross_cos_dis.T[i_][sorted_previous_advance_flag].sum(), \
                                                          sum(sorted_previous_advance_flag))
                
        for i_, fund_ in enumerate(sorted_current_funds):
            my_order_ = g_rank_raw_data[previous_date].get(fund_, -1.0)
            top10_num_ = int(math.ceil(g_funds_num[previous_date] * 0.1))
            top10_order_ = g_rank_value[previous_date][top10_num_]
            order_ = top10_order_ if top10_order_ > my_order_ else my_order_
            
            sorted_previous_advance_flag = [True if g_rank_raw_data[previous_date].get(foo_fund, -2.0) > order_ else False \
                                         for foo_fund in sorted_previous_funds]
            if any(sorted_previous_advance_flag):
                following_advance_10_sim_[fund_][current_date] = (cross_cos_dis.T[i_][sorted_previous_advance_flag].sum(), \
                                                          sum(sorted_previous_advance_flag))
                
        for i_, fund_ in enumerate(sorted_current_funds):
            my_order_ = g_rank_raw_data[previous_date].get(fund_, -1.0)
            top10_num_ = int(math.ceil(g_funds_num[previous_date] * 0.05))
            top10_order_ = g_rank_value[previous_date][top10_num_]
            order_ = top10_order_ if top10_order_ > my_order_ else my_order_
            
            sorted_previous_advance_flag = [True if g_rank_raw_data[previous_date].get(foo_fund, -2.0) > order_ else False \
                                         for foo_fund in sorted_previous_funds]
            if any(sorted_previous_advance_flag):
                following_advance_5_sim_[fund_][current_date] = (cross_cos_dis.T[i_][sorted_previous_advance_flag].sum(), \
                                                          sum(sorted_previous_advance_flag))

        for i_, fund_ in enumerate(sorted_current_funds):
            my_order_ = g_rank_raw_data[previous_date].get(fund_, -1.0)
            top10_num_ = int(math.ceil(g_funds_num[previous_date] * 0.15))
            top10_order_ = g_rank_value[previous_date][top10_num_]
            order_ = top10_order_ if top10_order_ > my_order_ else my_order_
            
            sorted_previous_advance_flag = [True if g_rank_raw_data[previous_date].get(foo_fund, -2.0) > order_ else False \
                                         for foo_fund in sorted_previous_funds]
            if any(sorted_previous_advance_flag):
                following_advance_15_sim_[fund_][current_date] = (cross_cos_dis.T[i_][sorted_previous_advance_flag].sum(), \
                                                          sum(sorted_previous_advance_flag))
                
            if fund_ == u'000001' and previous_date == datetime.datetime(2004,10,1):
                print sorted_previous_funds
                print sorted_previous_advance_flag
                print trade_vec[previous_date].keys()
                print [trade_vec[previous_date][funds_].sum() for funds_ in trade_vec[previous_date]]
                
                
    return leading_sim_, current_sim_, following_sim_, following_advance_sim_, \
                following_advance_5_sim_, following_advance_10_sim_, following_advance_15_sim_

g_leading_sim, g_current_sim, g_following_sim, g_following_advance_sim, \
    g_following_advance_5_sim, g_following_advance_10_sim, g_following_advance_15_sim = calc_sim3(g_trade_vector)

2004-01-01 00:00:00
2004-04-01 00:00:00
2004-07-01 00:00:00
2004-10-01 00:00:00
2005-01-01 00:00:00
2005-04-01 00:00:00
2005-07-01 00:00:00
2005-10-01 00:00:00
2006-01-01 00:00:00
2006-04-01 00:00:00
2006-07-01 00:00:00
2006-10-01 00:00:00
2007-01-01 00:00:00
2007-04-01 00:00:00
2007-07-01 00:00:00
2007-10-01 00:00:00
2008-01-01 00:00:00
2008-04-01 00:00:00
2008-07-01 00:00:00
2008-10-01 00:00:00
2009-01-01 00:00:00
2009-04-01 00:00:00
2009-07-01 00:00:00
2009-10-01 00:00:00
2010-01-01 00:00:00
2010-04-01 00:00:00
2010-07-01 00:00:00
2010-10-01 00:00:00
2011-01-01 00:00:00
2011-04-01 00:00:00
2011-07-01 00:00:00
2011-10-01 00:00:00
2012-01-01 00:00:00
2012-04-01 00:00:00
2012-07-01 00:00:00
2012-10-01 00:00:00
2013-01-01 00:00:00
2013-04-01 00:00:00
2013-07-01 00:00:00
2013-10-01 00:00:00
2014-01-01 00:00:00
2014-04-01 00:00:00
2014-07-01 00:00:00
2014-10-01 00:00:00
2015-01-01 00:00:00
2015-04-01 00:00:00
2015-07-01 00:00:00
2015-10-01 00:00:00
2016-01-01 00:00:00
2016-04-01 00:00:00


In [28]:
g_jingli_leading_sim = collections.defaultdict(lambda :collections.defaultdict(dict))
g_jingli_current_sim = collections.defaultdict(lambda :collections.defaultdict(dict))
g_jingli_following_sim = collections.defaultdict(lambda :collections.defaultdict(dict))
g_jingli_following_advance_sim = collections.defaultdict(lambda :collections.defaultdict(dict))
g_jingli_following_advance_5_sim = collections.defaultdict(lambda :collections.defaultdict(dict))
g_jingli_following_advance_10_sim = collections.defaultdict(lambda :collections.defaultdict(dict))
g_jingli_following_advance_15_sim = collections.defaultdict(lambda :collections.defaultdict(dict))

g_jingli_leading_sim_counter = 0
g_jingli_current_sim_counter = 0
g_jingli_following_sim_counter = 0
g_jingli_following_advance_sim_counter = 0
g_jingli_following_advance_5_sim_counter = 0
g_jingli_following_advance_10_sim_counter = 0
g_jingli_following_advance_15_sim_counter = 0


jingli_info = collections.defaultdict(lambda : collections.defaultdict(lambda :["",0]))

with xlrd.open_workbook(u"基金经理.xls") as wb:
    ws = wb.sheet_by_index(0)
    for r in xrange(ws.nrows):
        funds = ws.cell(r,0).value[:6]
        quater = datetime.datetime(int(ws.cell(r,4).value[:4]), 3*int(ws.cell(r,4).value[6])-2, 1)
        name = ws.cell(r,5).value
        num = int(ws.cell(r,6).value)
        if quater not in quater_date:
            print u"error at %s %s"%(funds, quater)
        elif num > jingli_info[funds][quater][1]:            
            jingli_info[funds][quater][0] = name
            jingli_info[funds][quater][1] = num

for funds in jingli_info:
    sorted_quater = sorted(jingli_info[funds].keys())
    start_idx = quater_date.index(sorted_quater[0])
    end_idx = quater_date.index(sorted_quater[-1])
    for idx in xrange(start_idx, end_idx+1):
        if quater_date[idx] not in jingli_info[funds]:
            jingli_info[funds][quater_date[idx]] = ["zhanwei", 1]
            
for funds in jingli_info:
    sorted_quater = sorted(jingli_info[funds].keys())
    for idx, quater in enumerate(sorted_quater):
        if idx == 0 or jingli_info[funds][sorted_quater[idx-1]][0] != jingli_info[funds][quater][0]:
            if funds in g_leading_sim and quater in g_leading_sim[funds]:                
                g_jingli_leading_sim_counter += 1
            if funds in g_current_sim and quater in g_current_sim[funds]:                    
                g_jingli_current_sim_counter += 1
            if funds in g_following_sim and quater in g_following_sim[funds]:                
                g_jingli_following_sim_counter += 1
            if funds in g_following_advance_sim and quater in g_following_advance_sim[funds]:                
                g_jingli_following_advance_sim_counter += 1
            if funds in g_following_advance_10_sim and quater in g_following_advance_10_sim[funds]:                
                g_jingli_following_advance_10_sim_counter += 1
            if funds in g_following_advance_5_sim and quater in g_following_advance_5_sim[funds]:                
                g_jingli_following_advance_5_sim_counter += 1
            if funds in g_following_advance_15_sim and quater in g_following_advance_15_sim[funds]:                
                g_jingli_following_advance_15_sim_counter += 1
        
        if funds in g_leading_sim and quater in g_leading_sim[funds]:   
            g_jingli_leading_sim[g_jingli_leading_sim_counter][funds][quater] = g_leading_sim[funds][quater]
        if funds in g_current_sim and quater in g_current_sim[funds]:
            g_jingli_current_sim[g_jingli_current_sim_counter][funds][quater] = g_current_sim[funds][quater]
        if funds in g_following_sim and quater in g_following_sim[funds]:  
            g_jingli_following_sim[g_jingli_following_sim_counter][funds][quater] = g_following_sim[funds][quater]
        if funds in g_following_advance_sim and quater in g_following_advance_sim[funds]:  
            g_jingli_following_advance_sim[g_jingli_following_advance_sim_counter][funds][quater] \
                = g_following_advance_sim[funds][quater]
        if funds in g_following_advance_10_sim and quater in g_following_advance_10_sim[funds]:  
            g_jingli_following_advance_10_sim[g_jingli_following_advance_10_sim_counter][funds][quater] \
                = g_following_advance_10_sim[funds][quater]
                
        if funds in g_following_advance_5_sim and quater in g_following_advance_5_sim[funds]:  
            g_jingli_following_advance_5_sim[g_jingli_following_advance_5_sim_counter][funds][quater] \
                = g_following_advance_5_sim[funds][quater]
                
        if funds in g_following_advance_15_sim and quater in g_following_advance_15_sim[funds]:  
            g_jingli_following_advance_15_sim[g_jingli_following_advance_15_sim_counter][funds][quater] \
                = g_following_advance_15_sim[funds][quater]

In [ ]:
a=u"2016年第1季度"
print jingli_info[u'000001'][datetime.datetime(2004,7,1)][0], jingli_info[u'000001'][datetime.datetime(2004,7,1)][1]
for seg_idx in g_jingli_following_advance_10_sim:
    for funds in g_jingli_following_advance_10_sim[seg_idx]:
        if funds == u'000001':
            sorted_quarter = sorted(g_jingli_following_advance_10_sim[seg_idx][funds].keys())
            print seg_idx, funds, sorted_quarter[0], sorted_quarter[-1], len(sorted_quarter)

In [29]:
workbook = xlsxwriter.Workbook(u'quarter_no_myself_distance_jingli.xlsx')
current_ws = workbook.add_worksheet(u"current")
leading_ws = workbook.add_worksheet(u"leading")
following_ws = workbook.add_worksheet(u"following")
following_advance_ws = workbook.add_worksheet(u"following_advance")
following_advance_5_ws = workbook.add_worksheet(u"following_advance_5")
following_advance_10_ws = workbook.add_worksheet(u"following_advance_10")
following_advance_15_ws = workbook.add_worksheet(u"following_advance_15")

def dump_result(ws, jingli_data):
    ws.write(0,0,u"fund")
    ws.write(0,1,u"quater")
    ws.write(0,2,u"sum_distance")
    ws.write(0,3,u"num")
    ws.write(0,4,u"avg_distance")
    ws.write(0,5,u"1-avg")
    ws.write(0,6,u"avg-4(include)")
    ws.write(0,7,u"avg-2(exclude)")
    ws.write(0,8,u"avg-4(exclude)")
    ws.write(0,9,u"avg-6(exclude)")
    
    wt_idx=1
    for seg, data in jingli_data.iteritems():
        for fund_ in data:
            sorted_quater = sorted(data[fund_].keys(), reverse=True)
            for i, quater_ in enumerate(sorted_quater):
                ws.write(wt_idx,0,fund_)
                ws.write(wt_idx,1,u"%dQ%d"%(quater_.year, quater_.month/3+1))
                ws.write(wt_idx,2,data[fund_][quater_][0])
                ws.write(wt_idx,3,data[fund_][quater_][1])
                ws.write(wt_idx,4,data[fund_][quater_][0]/data[fund_][quater_][1])
                ws.write(wt_idx,5,1-data[fund_][quater_][0]/data[fund_][quater_][1])

                total = 0.0
                end_idx = min((i+4, len(sorted_quater)))
                for j in xrange(i, end_idx):
                    total += (1-data[fund_][sorted_quater[j]][0]/data[fund_][sorted_quater[j]][1])
                ws.write(wt_idx,6,total/(end_idx-i))

                if i < len(sorted_quater)-1:
                    total = 0.0
                    end_idx = min((i+3, len(sorted_quater)))
                    for j in xrange(i+1, end_idx):
                        total += (1-data[fund_][sorted_quater[j]][0]/data[fund_][sorted_quater[j]][1])
                    ws.write(wt_idx,7,total/(end_idx-i-1))

                    total = 0.0
                    end_idx = min((i+5, len(sorted_quater)))
                    for j in xrange(i+1, end_idx):
                        total += (1-data[fund_][sorted_quater[j]][0]/data[fund_][sorted_quater[j]][1])
                    ws.write(wt_idx,8,total/(end_idx-i-1))

                    total = 0.0
                    end_idx = min((i+7, len(sorted_quater)))
                    for j in xrange(i+1, end_idx):
                        total += (1-data[fund_][sorted_quater[j]][0]/data[fund_][sorted_quater[j]][1])
                    ws.write(wt_idx,9,total/(end_idx-i-1))
                else:
                    ws.write(wt_idx,7,u'.')
                    ws.write(wt_idx,8,u'.')
                    ws.write(wt_idx,9,u'.')

                wt_idx += 1 
            
dump_result(current_ws, g_jingli_current_sim)
dump_result(leading_ws, g_jingli_leading_sim)
dump_result(following_ws, g_jingli_following_sim)
dump_result(following_advance_ws, g_jingli_following_advance_sim)
dump_result(following_advance_5_ws, g_jingli_following_advance_5_sim)
dump_result(following_advance_10_ws, g_jingli_following_advance_10_sim)
dump_result(following_advance_15_ws, g_jingli_following_advance_15_sim)



workbook.close()

In [ ]:
print g_following_advance_sim[u'110002'][datetime.datetime(2004,10,1)]

In [ ]:
g_leading_raw_data = collections.defaultdict(list)
g_leading_rank_data = {}
for fund_name in g_leading_sim:
    for quarter in  g_leading_sim[fund_name]:
        g_leading_raw_data[quarter].append((fund_name, \
                                           1-g_leading_sim[fund_name][quarter][0]/g_leading_sim[fund_name][quarter][1]))
        
for quarter in g_leading_raw_data:
    g_leading_rank_data[quarter] = sorted(g_leading_raw_data[quarter], key=lambda var: var[1])

for quarter in g_leading_rank_data:
    rank_num = int(math.ceil(g_funds_num[quarter] * 0.1))
    leading_set = set(g_leading_rank_data[quarter][i][0] for i in xrange(rank_num))
    rank_set = set(g_sorted_fund[i] for i in xrange(len(g_sorted_fund)) if g_rank[quarter][i] < rank_num)
    
    print quarter, len(leading_set&rank_set) / float(len(leading_set|rank_set))
    
#print g_leading_raw_data[datetime.datetime(2004,1,1)]
#print g_leading_rank_data[datetime.datetime(2004,1,1)]
